# PyOD - IsolationForest

## Carga de datos

Cargamos librerías y datos:

Las típicas (pandas, matplotlib, numpy)...

Funciones de sklearn de preprocesado y métricas.

Modelos y métricas de PyOD.

In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score

from pyod.models.iforest import IForest

random_state = np.random.RandomState(42)

In [2]:
df = pd.read_csv('../../Datasets/Dataset_2.csv',index_col='fecha', usecols=['fecha','FormacionNIRHumedadPV', 'Etapa2MWHumedadPV','Negro', 'CurvaCola', 'Congelado', 'Hum'])
df['Diferencia'] = df['FormacionNIRHumedadPV'] - df['Etapa2MWHumedadPV']
df = df.drop(['FormacionNIRHumedadPV','Etapa2MWHumedadPV'], axis=1)
df = df.loc[:,['Diferencia','Negro', 'CurvaCola', 'Congelado', 'Hum']]
df.index = pd.to_datetime(df.index)
df.head(2)

,Diferencia,Negro,CurvaCola,Congelado,Hum
fecha,,,,,
2021-02-10 10:00:00,-13.141570,1,0,1,0
2021-02-10 10:01:00,-12.412745,1,0,1,0


Unimos todas las anomalías en una columna, para posteriormente realizar las métricas tanto para todas las anomalías como para cada una en concreto.

Para ello, sumo las columnas de anomalías, y después las que son mayores que cero las establezco como uno (porque significa que alguna de las columnas si tenía anomalía registrada), las demás como cero.

In [3]:
df['Anomalia'] = df['Negro'] + df['Congelado'] + df['Hum']
df['Anomalia'] = df['Anomalia'].map(lambda x: 1 if x!=0 else 0)

df = df.drop(df[df['Anomalia']==1].index)
df = df.drop(['Negro','Congelado','Hum','Anomalia'], axis=1)

lista_anomalias = ['CurvaCola']

atributos = df.columns.drop(lista_anomalias)
len(atributos)

1

## Escogemos un subconjunto del dataset para entrenamientos más cortos

In [4]:
#df = df.iloc[3000:60000,:]

## Preparación

Separo conjuntos de train, validation y test, y estandarizo:

Separo los atributos en X y las anomalias en Y. De esta manera, al realizar el train_test_split, se mantendrán las proporciones de cada anomalía, con muestreos temporales aleatorios.

Primero separo en train-test (80-20) y después separo el test en test-validation (50-50), para así obtener finalmente train-validation-test (80-10-10).

Una vez separado, entreno el StandardScaler() con el conjunto de entrenamiento, y se lo aplico al conjunto de validación y test.

In [5]:
#Separo los atributos para el entrenamiento de la salida
X = df.loc[:, atributos]
Y = df.loc[:, lista_anomalias]

#Calculo la proporcion de outliers presentes
proporcion_outliers = round(np.count_nonzero(Y) / len(Y),3)

#Separo entrenamiento y test (80-20)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = random_state)

#Normalizo
standarizer = StandardScaler()
standarizer.fit(X_train)
X_train_standarized = standarizer.transform(X_train)
X_test_standarized = standarizer.transform(X_test)

## Entrenamiento

Para ello, establezco primero la lista de hiperparámetros a entrenar y la de métricas a analizar.

creo dos arrays de ceros, uno para las scores y otro para las labels.

Realizo un bucle, estableciendo un entrenamiento por cada hiperparámetro.

Después, extraigo scores y labels, almacenándolas en un array, y calculo las métricas, haciendo lo mismo. Una vez termina el bucle, estos dos vectores los paso a un dataframe para su visualización.

In [6]:
t0 = time()

#Establecemos parametros
metricas_list = ['roc_auc','accuracy','precision','kappa','sensibilidad','especificidad']
anomalia = 'CurvaCola'

#Entrenamiento
n = 500
clf = IForest(n_estimators = n, n_jobs = -1, random_state = random_state)
clf.fit(X_train_standarized)

#Prediccion
Y_pred = clf.predict(X_test_standarized)

#Metricas
roc_auc = roc_auc_score(Y_test[anomalia], Y_pred)
accuracy = accuracy_score(Y_test[anomalia],Y_pred)
precision = precision_score(Y_test[anomalia],Y_pred)
kappa = cohen_kappa_score(Y_test[anomalia],Y_pred)
sensibilidad = recall_score(Y_test[anomalia],Y_pred)
especificidad = recall_score(Y_test[anomalia],Y_pred, pos_label=0)

valores = [roc_auc,accuracy,precision,kappa,sensibilidad,especificidad]
metricas = pd.DataFrame(valores)
metricas.index = metricas_list
metricas.columns = [anomalia]

#Tiempo
t1 = time()
duration = round(t1 - t0, ndigits=4)
print('Tiempo: ', duration)

Tiempo:  61.5247


Analizo las métricas:

In [7]:
metricas.T

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
CurvaCola,0.487704,0.82788,0.067756,-0.023489,0.0748,0.900608


In [8]:
Y_test['Y_pred'] = Y_pred
Y_test.to_csv('Resultados/PyOD_IForest_completo.csv')